In [1]:
import pandas as pd
import geopandas as gpd
from os import path, makedirs
from unidecode import unidecode

In [2]:
_url_dist_mun = (
    "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:distrito_municipal&outputFormat=application/json"
)
_url_subs = (
    "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:subprefeitura&outputFormat=application/json"
)
_url_dist_censo = (
    "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/gpkg/UF/SP/SP_distritos_CD2022.gpkg"
)

### Indicador domicilios não ocupados (censo 2022)

In [3]:
if "dist_mun" not in globals():
    _url_dist_mun = "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:distrito_municipal&outputFormat=application/json"
    dist_mun = gpd.read_file(_url_dist_mun)

In [4]:

if "subs" not in globals():
    _url_subs = "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:subprefeitura&outputFormat=application/json"
    subs = gpd.read_file(_url_subs)

In [5]:

if "dist_censo" not in globals():
    _url_dist_censo = "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/gpkg/UF/SP/SP_distritos_CD2022.gpkg"
    dist_censo = gpd.read_file(_url_dist_censo)

In [6]:
dist_censo = dist_censo[dist_censo["CD_MUN"] == "3550308"].copy()
dist_censo["nm_distrito_municipal"] = dist_censo["NM_DIST"].apply(unidecode).str.upper()

In [7]:
dist_mun = dist_mun.merge(
    dist_censo[["nm_distrito_municipal", "v0003", "v0007"]],
    on="nm_distrito_municipal",
    how="left"
)

In [8]:
dist_mun = dist_mun.rename(columns={
    "v0003": "total_domicilios_particulares",
    "v0007": "domicilios_particulares_ocupados"
}).astype({
    "total_domicilios_particulares": "Int64",
    "domicilios_particulares_ocupados": "Int64"
})


In [9]:
dist_mun["domicilios_particulares_nao_ocupados"] = (
    dist_mun["total_domicilios_particulares"] - dist_mun["domicilios_particulares_ocupados"]
)

In [10]:

dist_mun["cd_subprefeitura"] = dist_mun["cd_identificador_subprefeitura"].apply(
    lambda cd: f"0{int(cd)}" if int(cd) < 10 else str(int(cd))
)


In [11]:

dom_subs = dist_mun.groupby("cd_subprefeitura", as_index=False).agg({
    "total_domicilios_particulares": "sum",
    "domicilios_particulares_ocupados": "sum",
    "domicilios_particulares_nao_ocupados": "sum"
})

In [12]:
subs_out = subs.merge(dom_subs, on="cd_subprefeitura", how="left")

In [13]:
indicador_domicilios_nao_ocupados_df = subs_out[[
    "cd_subprefeitura",
    "nm_subprefeitura",
    "domicilios_particulares_nao_ocupados",
    "total_domicilios_particulares",
    "domicilios_particulares_ocupados"
]].copy()

In [14]:
indicador_domicilios_nao_ocupados_df

,cd_subprefeitura,nm_subprefeitura,domicilios_particulares_nao_ocupados,total_domicilios_particulares,domicilios_particulares_ocupados
0,16,CIDADE ADEMAR,19191,167686,148495
1,25,MOOCA,31094,181383,150289
2,13,IPIRANGA,29555,213697,184142
3,08,LAPA,21194,162783,141589
4,15,JABAQUARA,13359,96239,82880
5,28,GUAIANASES,12208,106978,94770
6,14,SANTO AMARO,17893,128102,110209
7,32,SAPOPEMBA,10854,105558,94704
8,23,SAO MIGUEL,13826,133575,119749
9,06,JACANA-TREMEMBE,13860,112875,99015


### Indicador Estimativa de domicílios em favela (%)

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [15]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

for nome, df in [
    ('indicador_domicilios_nao_ocupados', indicador_domicilios_nao_ocupados_df),
]:
    filename = path.join(output_dir, nome)
    df.to_csv(f'{filename}.csv', sep=';', decimal=',', encoding='utf8', index=False)
